In [ ]:
import pandas as pd

In [9]:
file_path = 'data.csv'
df = pd.read_csv('data.csv')

print(df.head())
print(df.columns)

    Rk  Gcar  Gtm        Date Team Unnamed: 5  Opp          Result GS     MP  \
0  1.0   1.0  1.0  2006-04-22  CLE        NaN  WAS         W 97-86  *  48:00   
1  2.0   2.0  2.0  2006-04-25  CLE        NaN  WAS         L 84-89  *  43:28   
2  3.0   3.0  3.0  2006-04-28  CLE          @  WAS         W 97-96  *  47:37   
3  4.0   4.0  4.0  2006-04-30  CLE          @  WAS        L 96-106  *  45:20   
4  5.0   5.0  5.0  2006-05-03  CLE        NaN  WAS  W 121-120 (OT)  *  46:07   

   ...  DRB   TRB   AST  STL  BLK   TOV   PF   PTS  GmSc   +/-  
0  ...  8.0  11.0  11.0  0.0  0.0   4.0  3.0  32.0  23.3  11.0  
1  ...  7.0   9.0   2.0  2.0  3.0  10.0  5.0  26.0   6.7  -2.0  
2  ...  4.0   5.0   3.0  2.0  0.0   4.0  3.0  41.0  27.4   1.0  
3  ...  5.0   6.0   5.0  0.0  0.0   7.0  5.0  38.0  23.0 -16.0  
4  ...  2.0   7.0   6.0  2.0  0.0   4.0  5.0  45.0  38.4  -4.0  

[5 rows x 34 columns]
Index(['Rk', 'Gcar', 'Gtm', 'Date', 'Team', 'Unnamed: 5', 'Opp', 'Result',
       'GS', 'MP', 'FG', 'FGA',

In [12]:
import requests
from lxml import html

# Load the page
url = "https://www.asapsports.com/show_player.php?id=13888"
response = requests.get(url)
tree = html.fromstring(response.content)

# Get all interview links from the 3rd <td> in each row of the main interview table
interview_links = tree.xpath('//table[1]//tr/td[3]/b/a')

interviews = []
for link in interview_links:
    href = link.get('href')
    text = link.text_content().strip()
    if href:
        full_url = f"https://www.asapsports.com/{href}"
        interviews.append({'title': text, 'url': full_url})

# Print all links
for interview in interviews:
    print(f"{interview['title']}: {interview['url']}")


NBA ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=204675
ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=194257
ALL-STAR GAME: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=194258
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193132
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193098
NBA IN-SEASON TOURNAMENT: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=193082
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=187284
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interview.php?id=187224
NBA WESTERN CONFERENCE FINALS: LAKERS VS. NUGGETS: https://www.asapsports.com/http://www.asapsports.com/show_interv

In [50]:
import time
import json
import requests
from lxml import html
from urllib.parse import urljoin

def fetch_last_n_interviews(player_url, n=10, pause=1.0):
    # Fetch player page and extract interview links
    resp = requests.get(player_url)
    resp.raise_for_status()
    tree = html.fromstring(resp.content)
    
    raw_links = tree.xpath('//table[1]//tr/td[3]/b/a')[:]
    interviews = [{
        'title': a.text_content().strip(),
        'url': urljoin(player_url, a.get('href').strip())
    } for a in raw_links if a.get('href')]

    results = []
    for iv in interviews:
        r = requests.get(iv['url'])
        r.raise_for_status()
        page = html.fromstring(r.content)
        
        # Extract date
        date = page.xpath('string(//h2)').strip() or "Unknown Date"
        qa = []

        # Strategy 1: Handle questions in <b> tags with answers in siblings
        container = page.xpath('//td[@valign="top"]')
        if container:
            container = container[0]
            # Find all elements containing questions (flexible matching)
            question_nodes = container.xpath('.//*[contains(., "Q.") or starts-with(normalize-space(), "Q.")]')
            
            for idx, node in enumerate(question_nodes):
                question_text = node.text_content().replace('Â', '').replace('\xa0', ' ').strip()
                if question_text.startswith('Q. '):
                    question_text = question_text[3:].strip()
                
                answer_parts = []
                current_node = node.getnext()
                next_q = question_nodes[idx+1] if (idx+1) < len(question_nodes) else None

                while current_node is not None and current_node != next_q:
                    if isinstance(current_node, html.HtmlElement):
                        txt = current_node.text_content().replace('Â', '').strip()
                        if txt and not txt.startswith('Q.'):
                            answer_parts.append(txt)
                    current_node = current_node.getnext()
                
                if not answer_parts:
                    node_text = node.text_content().replace('Â', '').split('Q.')[-1].strip()
                    if node_text:
                        answer_parts.append(node_text)

                qa.append({
                    'question': f"Q. {question_text}" if not question_text.startswith('Q.') else question_text,
                    'answer': ' '.join(answer_parts).strip()
                })

        # Strategy 2: Handle questions split across multiple <p> tags
        if not qa:
            paras = page.xpath('//p')
            for idx, p in enumerate(paras):
                txt = p.text_content().replace('Â', '').strip()
                if txt.startswith('Q.') or txt.startswith('Q. '):
                    question = txt.split('Q.')[-1].strip()
                    answer = []
                    for sib in paras[idx+1:]:
                        sib_txt = sib.text_content().replace('Â', '').strip()
                        if sib_txt.startswith('Q.'):
                            break
                        if sib_txt:
                            answer.append(sib_txt)
                    
                    qa.append({
                        'question': f"Q. {question}",
                        'answer': ' '.join(answer).strip()
                    })

        # Strategy 3: Raw text fallback
        if not qa:
            pre_text = page.xpath('//pre//text()')
            if pre_text:
                cleaned = [t.replace('Â', '').strip() for t in pre_text if t.strip()]
                qa = [{'question': None, 'answer': ' '.join(cleaned)}]

        # Strategy 4: Specifically handle <b> with " Q." and get sibling text
        if not qa and container is not None:
            b_nodes = container.xpath('.//b')
            for idx, b in enumerate(b_nodes):
                text = b.text_content().replace('Â', '').replace('\xa0', ' ').strip()
                if text.startswith("Q. ") or text.startswith(" Q."):
                    question = text.lstrip(" Q.").strip()

                    # Find the immediate text node that follows (answer)
                    parent = b.getparent()
                    if parent is not None:
                        all_children = list(parent)
                        try:
                            b_index = all_children.index(b)
                            answer_parts = []
                            for sib in all_children[b_index+1:]:
                                sib_text = sib.text_content().strip()
                                if sib_text.startswith("Q."):
                                    break
                                if sib_text:
                                    answer_parts.append(sib_text)
                            if not answer_parts and parent.tail:
                                answer_parts.append(parent.tail.strip())

                            if answer_parts:
                                qa.append({
                                    'question': f"Q. {question}",
                                    'answer': ' '.join(answer_parts).strip()
                                })
                        except ValueError:
                            continue

        # Strategy 5: Handle " Q." in <b> and get answer from next text node (like in image)
        if not qa and container is not None:
            b_nodes = container.xpath('.//b')
            for b in b_nodes:
                text = b.text_content().replace('\xa0', ' ').strip()
                if text.startswith("Q.") or text.startswith(" Q."):
                    question = text.lstrip(" Q.").strip()

                    # Attempt to grab the immediately following text node (not another tag)
                    parent = b.getparent()
                    if parent is not None:
                        # Get the full text content of the parent <td>, then split on the question
                        full_text = parent.text_content().replace('\xa0', ' ')
                        parts = full_text.split(text, 1)
                        if len(parts) == 2:
                            answer_text = parts[1].strip()
                            if answer_text:
                                qa.append({
                                    'question': f"Q. {question}",
                                    'answer': answer_text
                                })

        results.append({
            'title': iv['title'],
            'url': iv['url'],
            'date': date,
            'qa': qa
        })
        time.sleep(pause)

    return results

if __name__ == '__main__':
    transcripts = fetch_last_n_interviews('http://www.asapsports.com/show_player.php?id=13888', n=10)
    with open('lebron_interviews.json', 'w', encoding='utf-8') as f:
        json.dump(transcripts, f, indent=2, ensure_ascii=False)
    print("✅ Fixed transcripts saved to lebron_interviews.json")


✅ Fixed transcripts saved to lebron_interviews.json
